In [142]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

In [144]:
df = pd.read_csv("vacancies.csv") 
df

,id,title,company_name,currency,experience,type_of_employment,work_format,description,skills,address,published_at,archived,url,min_experience,max_experience,salary_to,salary_from,russian_salary_to,russian_salary_from
0,112352524,Системный аналитик,Медиа Сервис,NaN,От 3 до 6 лет,Полная занятость,Удаленная работа,<p>Мы – современная аккредитованная IT- компан...,{},Санкт-Петербург,2024-12-02 17:36:51,False,https://hh.ru/vacancy/112352524,3,6.0,NaN,NaN,NaN,NaN
1,110739510,Ведущий аналитик данных,Банк Синара,NaN,От 1 года до 3 лет,Полная занятость,Полный день,<p><strong>АО Банк Синара</strong> финансовая ...,"{SQL,Python,""Визуализация данных"",""Интерпретац...",Челябинск,2024-12-23 15:08:11,False,https://hh.ru/vacancy/110739510,1,3.0,NaN,NaN,NaN,NaN
2,111973043,Системный аналитик (Intelligent Document Proce...,СБЕР,NaN,От 3 до 6 лет,Полная занятость,Полный день,<p>Мы разрабатываем продукты для решения задач...,{},Москва,2024-12-24 14:30:29,False,https://hh.ru/vacancy/111973043,3,6.0,NaN,NaN,NaN,NaN
3,112178936,Продуктовый аналитик,Lamoda Tech,NaN,От 1 года до 3 лет,Полная занятость,Полный день,<p>Мы в поиске <strong>Продуктового аналитика<...,"{SQL,Python,""Power BI"",""A/B тесты""}",Москва,2024-12-02 13:20:42,False,https://hh.ru/vacancy/112178936,1,3.0,NaN,NaN,NaN,NaN
4,112550498,Старший Аналитик,HR-Stalker,USD,От 3 до 6 лет,Полная занятость,Удаленная работа,<p><strong>Improvado</strong><br />Американска...,"{SQL,Clickhouse,BigQuery,Databases,""Моделирова...",Москва,2024-12-04 21:30:19,False,https://hh.ru/vacancy/112550498,3,6.0,5200.0,4500.0,459613.44,397742.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,112543174,Экономист-аналитик,ТД «Рос-Пак»,RUR,От 1 года до 3 лет,Полная занятость,Удаленная работа,<p>В связи с расширение производства и выходом...,"{""Работа с большим объемом информации"",Бюджети...",Краснодар,2024-12-04 17:40:16,False,https://hh.ru/vacancy/112543174,1,3.0,80000.0,NaN,80000.00,NaN
5572,107686857,Аналитик,СГМК,RUR,Нет опыта,Полная занятость,Полный день,<p><strong>Мы - Сибирская горно-металлургическ...,"{Ответственность,Внимательность,Коммуникабельн...",Новокузнецк,2024-12-19 08:45:35,False,https://hh.ru/vacancy/107686857,0,0.0,NaN,60000.0,NaN,60000.0
5573,112281035,Системный аналитик,Инновационный центр КАМАЗ,RUR,От 3 до 6 лет,Полная занятость,Полный день,<p><strong>ООО «Инновационный центр «КАМАЗ»</s...,{},Москва,2024-12-01 18:48:11,False,https://hh.ru/vacancy/112281035,3,6.0,NaN,200000.0,NaN,200000.0
5574,115416164,Химик-аналитик в Фруктис (ст.Старовеличковская),АПК Астраханский,RUR,От 1 года до 3 лет,Полная занятость,Сменный график,<strong>Обязанности:</strong> <ul> <li>Проведе...,{},Тимашевск,2025-01-13 14:18:50,False,https://hh.ru/vacancy/115416164,1,3.0,NaN,55000.0,NaN,55000.0


In [145]:
missing = (df.isna().sum() / len(df)) * 100
missing = missing[missing > 0].sort_values(ascending=False)
missing = pd.DataFrame({'Процент пропущенных значений' : missing})
missing

,Процент пропущенных значений
salary_to,84.433286
russian_salary_to,84.433286
salary_from,74.121234
russian_salary_from,74.121234
currency,69.996413
max_experience,2.295552


In [146]:
# Первичный анализ
print(df.info())
print(df.describe())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5576 entries, 0 to 5575
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   5576 non-null   int64  
 1   title                5576 non-null   object 
 2   company_name         5576 non-null   object 
 3   currency             1673 non-null   object 
 4   experience           5576 non-null   object 
 5   type_of_employment   5576 non-null   object 
 6   work_format          5576 non-null   object 
 7   description          5576 non-null   object 
 8   skills               5576 non-null   object 
 9   address              5576 non-null   object 
 10  published_at         5576 non-null   object 
 11  archived             5576 non-null   bool   
 12  url                  5576 non-null   object 
 13  min_experience       5576 non-null   int64  
 14  max_experience       5448 non-null   float64
 15  salary_to            868 non-null    f

In [147]:
# Создаем целевую переменную
df['salary'] = np.where(
    df['russian_salary_from'].notna() & df['russian_salary_to'].notna(),
    (df['russian_salary_from'] + df['russian_salary_to']) / 2,
    np.where(
        df['russian_salary_from'].notna(),
        df['russian_salary_from'],
        df['russian_salary_to']
    )
)

# Удаляем строки, где salary = NaN (нет данных о зарплате)
df = df.dropna(subset=['salary'])

In [152]:
print(df.info())
print(df.describe())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 1673 entries, 4 to 5575
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   1673 non-null   int64  
 1   title                1673 non-null   object 
 2   company_name         1673 non-null   object 
 3   currency             1673 non-null   object 
 4   experience           1673 non-null   object 
 5   type_of_employment   1673 non-null   object 
 6   work_format          1673 non-null   object 
 7   description          1673 non-null   object 
 8   skills               1673 non-null   object 
 9   address              1673 non-null   object 
 10  published_at         1673 non-null   object 
 11  archived             1673 non-null   bool   
 12  url                  1673 non-null   object 
 13  min_experience       1673 non-null   int64  
 14  max_experience       1643 non-null   float64
 15  salary_to            868 non-null    float6

In [153]:
df.columns

Index(['id', 'title', 'company_name', 'currency', 'experience',
       'type_of_employment', 'work_format', 'description', 'skills', 'address',
       'published_at', 'archived', 'url', 'min_experience', 'max_experience',
       'salary_to', 'salary_from', 'russian_salary_to', 'russian_salary_from',
       'salary'],
      dtype='object')

In [156]:
df = df.drop(columns=[
    'id', 'company_name', 'currency', 'published_at', 
    'archived', 'url', 'salary_to', 'salary_from', 'experience', 'russian_salary_to', 'russian_salary_from'
])

In [158]:
df.columns

Index(['title', 'type_of_employment', 'work_format', 'description', 'skills',
       'address', 'min_experience', 'max_experience', 'salary'],
      dtype='object')

In [160]:
df = df.drop(columns=['description'
])

In [162]:
print(df[['min_experience', 'max_experience']].corr())

                min_experience  max_experience
min_experience        1.000000        0.971802
max_experience        0.971802        1.000000


Можно оставить только max_experience, так как корреляция между этими признаками очень высокая

In [165]:
print(df[['min_experience', 'max_experience', 'salary']].corr())

                min_experience  max_experience    salary
min_experience        1.000000        0.971802  0.550029
max_experience        0.971802        1.000000  0.522721
salary                0.550029        0.522721  1.000000


In [167]:
df = df.drop(columns=['max_experience'
])

In [169]:
df = pd.get_dummies(df, columns=['type_of_employment', 'work_format'])

In [171]:
import re

def clean_skills(skill_str):
    # Удаляем фигурные скобки
    skill_str = skill_str.strip('{}')
    # Удаляем кавычки и лишние пробелы
    skills = re.findall(r'(?:[^,"]|"(?:\\.|[^"])*")+', skill_str)
    skills = [s.strip().strip('"') for s in skills if s.strip()]
    return skills

# Пример применения
df['skills_cleaned'] = df['skills'].apply(clean_skills)

In [173]:
df

,title,skills,address,min_experience,salary,type_of_employment_Полная занятость,type_of_employment_Проектная работа,type_of_employment_Стажировка,type_of_employment_Частичная занятость,work_format_Вахтовый метод,work_format_Гибкий график,work_format_Полный день,work_format_Сменный график,work_format_Удаленная работа,skills_cleaned
4,Старший Аналитик,"{SQL,Clickhouse,BigQuery,Databases,""Моделирова...",Москва,3,428677.92,True,False,False,False,False,False,False,False,True,"[SQL, Clickhouse, BigQuery, Databases, Моделир..."
18,Ведущий Аналитик (Продукт ETL),"{SQL,Clickhouse,BigQuery,Databases,""Моделирова...",Москва,3,375645.60,True,False,False,False,False,False,False,False,True,"[SQL, Clickhouse, BigQuery, Databases, Моделир..."
24,Продуктовый аналитик (ETL),"{SQL,Clickhouse,BigQuery,Databases,""Моделирова...",Москва,3,486129.60,True,False,False,False,False,False,False,False,True,"[SQL, Clickhouse, BigQuery, Databases, Моделир..."
25,Аналитик данных / Data Analyst / Data Scientist,"{Python,pandas,Regex,""Natural Language Process...",Санкт-Петербург,1,132580.80,False,False,False,True,False,False,False,False,True,"[Python, pandas, Regex, Natural Language Proce..."
27,Продуктовый аналитик / Data Analyst (junior),"{Python,SQL,""Анализ данных"",Amplitude,""Английс...",Санкт-Петербург,1,70709.76,True,False,False,False,False,False,False,False,True,"[Python, SQL, Анализ данных, Amplitude, Англий..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,Экономист-аналитик,"{""Работа с большим объемом информации"",Бюджети...",Краснодар,1,80000.00,True,False,False,False,False,False,False,False,True,"[Работа с большим объемом информации, Бюджетир..."
5572,Аналитик,"{Ответственность,Внимательность,Коммуникабельн...",Новокузнецк,0,60000.00,True,False,False,False,False,False,True,False,False,"[Ответственность, Внимательность, Коммуникабел..."
5573,Системный аналитик,{},Москва,3,200000.00,True,False,False,False,False,False,True,False,False,[]
5574,Химик-аналитик в Фруктис (ст.Старовеличковская),{},Тимашевск,1,55000.00,True,False,False,False,False,False,False,True,False,[]


In [175]:
from collections import Counter

# Собираем все навыки в один список
all_skills = [skill for sublist in df['skills_cleaned'] for skill in sublist]

# Топ-20 навыков
top_skills = [skill for skill, count in Counter(all_skills).most_common(20)]
print("Топ-20 навыков:", top_skills)

Топ-20 навыков: ['Аналитическое мышление', 'SQL', 'MS Excel', 'Анализ данных', 'Работа с большим объемом информации', 'Бизнес-анализ', 'Аналитика', 'Системный анализ', 'BPMN', 'Python', 'Аналитические исследования', 'UML', 'Разработка технических заданий', 'Работа с базами данных', 'Power BI', 'Excel', 'MS PowerPoint', 'Постановка задач разработчикам', 'Многозадачность', 'Финансовый анализ']


In [177]:
for skill in top_skills:
    df[f'skill_{skill}'] = df['skills_cleaned'].apply(lambda x: 1 if skill in x else 0)

# Удаляем промежуточные столбцы
df = df.drop(columns=['skills', 'skills_cleaned'])

In [179]:
df.columns

Index(['title', 'address', 'min_experience', 'salary',
       'type_of_employment_Полная занятость',
       'type_of_employment_Проектная работа', 'type_of_employment_Стажировка',
       'type_of_employment_Частичная занятость', 'work_format_Вахтовый метод',
       'work_format_Гибкий график', 'work_format_Полный день',
       'work_format_Сменный график', 'work_format_Удаленная работа',
       'skill_Аналитическое мышление', 'skill_SQL', 'skill_MS Excel',
       'skill_Анализ данных', 'skill_Работа с большим объемом информации',
       'skill_Бизнес-анализ', 'skill_Аналитика', 'skill_Системный анализ',
       'skill_BPMN', 'skill_Python', 'skill_Аналитические исследования',
       'skill_UML', 'skill_Разработка технических заданий',
       'skill_Работа с базами данных', 'skill_Power BI', 'skill_Excel',
       'skill_MS PowerPoint', 'skill_Постановка задач разработчикам',
       'skill_Многозадачность', 'skill_Финансовый анализ'],
      dtype='object')

In [181]:
print(df.isnull().sum())

title                                        0
address                                      0
min_experience                               0
salary                                       0
type_of_employment_Полная занятость          0
type_of_employment_Проектная работа          0
type_of_employment_Стажировка                0
type_of_employment_Частичная занятость       0
work_format_Вахтовый метод                   0
work_format_Гибкий график                    0
work_format_Полный день                      0
work_format_Сменный график                   0
work_format_Удаленная работа                 0
skill_Аналитическое мышление                 0
skill_SQL                                    0
skill_MS Excel                               0
skill_Анализ данных                          0
skill_Работа с большим объемом информации    0
skill_Бизнес-анализ                          0
skill_Аналитика                              0
skill_Системный анализ                       0
skill_BPMN   

In [183]:
top_titles = df['title'].value_counts().head(10).index
df['title_group'] = df['title'].apply(lambda x: x if x in top_titles else 'other')
df = pd.get_dummies(df, columns=['title_group'], prefix='title')
df = df.drop(columns=['title'])

In [185]:
df

,address,min_experience,salary,type_of_employment_Полная занятость,type_of_employment_Проектная работа,type_of_employment_Стажировка,type_of_employment_Частичная занятость,work_format_Вахтовый метод,work_format_Гибкий график,work_format_Полный день,...,title_Аналитик,title_Аналитик 1С,title_Аналитик в команду WFM,title_Аналитик данных,title_Бизнес-аналитик,title_Маркетолог-аналитик,title_Менеджер проектов в сфере медиааналитики,title_Системный аналитик,title_Финансовый аналитик,title_Экономист-аналитик
4,Москва,3,428677.92,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18,Москва,3,375645.60,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
24,Москва,3,486129.60,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
25,Санкт-Петербург,1,132580.80,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
27,Санкт-Петербург,1,70709.76,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,Краснодар,1,80000.00,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
5572,Новокузнецк,0,60000.00,True,False,False,False,False,False,True,...,True,False,False,False,False,False,False,False,False,False
5573,Москва,3,200000.00,True,False,False,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
5574,Тимашевск,1,55000.00,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [187]:
top_address = df['address'].value_counts().head(10).index
df['address_group'] = df['address'].apply(lambda x: x if x in top_address else 'other')
df = pd.get_dummies(df, columns=['address_group'], prefix='address')
df = df.drop(columns=['address'])

In [189]:
df

,min_experience,salary,type_of_employment_Полная занятость,type_of_employment_Проектная работа,type_of_employment_Стажировка,type_of_employment_Частичная занятость,work_format_Вахтовый метод,work_format_Гибкий график,work_format_Полный день,work_format_Сменный график,...,address_Владивосток,address_Екатеринбург,address_Казань,address_Краснодар,address_Москва,address_Нижний Новгород,address_Новосибирск,address_Ростов-на-Дону,address_Самара,address_Санкт-Петербург
4,3,428677.92,True,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
18,3,375645.60,True,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
24,3,486129.60,True,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
25,1,132580.80,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
27,1,70709.76,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,1,80000.00,True,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
5572,0,60000.00,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
5573,3,200000.00,True,False,False,False,False,False,True,False,...,False,False,False,False,True,False,False,False,False,False
5574,1,55000.00,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [191]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1673 entries, 4 to 5575
Data columns (total 53 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   min_experience                                  1673 non-null   int64  
 1   salary                                          1673 non-null   float64
 2   type_of_employment_Полная занятость             1673 non-null   bool   
 3   type_of_employment_Проектная работа             1673 non-null   bool   
 4   type_of_employment_Стажировка                   1673 non-null   bool   
 5   type_of_employment_Частичная занятость          1673 non-null   bool   
 6   work_format_Вахтовый метод                      1673 non-null   bool   
 7   work_format_Гибкий график                       1673 non-null   bool   
 8   work_format_Полный день                         1673 non-null   bool   
 9   work_format_Сменный график                    

In [195]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Выбираем только числовые признаки (исключаем бинарные bool)
numeric_cols = ['min_experience'] + [col for col in df.columns if col.startswith('skill_')]
X_numeric = df[numeric_cols]

# Считаем VIF
vif_data = pd.DataFrame()
vif_data["feature"] = X_numeric.columns
vif_data["VIF"] = [variance_inflation_factor(X_numeric.values, i) for i in range(X_numeric.shape[1])]
print(vif_data.sort_values("VIF", ascending=False))

                                      feature       VIF
19                      skill_Многозадачность  3.544144
16                                skill_Excel  2.990832
12                                  skill_UML  2.507164
9                                  skill_BPMN  2.472954
2                                   skill_SQL  2.201728
11           skill_Аналитические исследования  2.129169
7                             skill_Аналитика  2.042701
4                         skill_Анализ данных  1.839363
1                skill_Аналитическое мышление  1.836124
5   skill_Работа с большим объемом информации  1.793350
8                      skill_Системный анализ  1.662548
13       skill_Разработка технических заданий  1.524623
10                               skill_Python  1.467044
6                         skill_Бизнес-анализ  1.400010
3                              skill_MS Excel  1.380650
18       skill_Постановка задач разработчикам  1.360791
0                              min_experience  1

In [201]:
import statsmodels.api as sm

# Добавляем константу и преобразуем bool в int (для statsmodels)
X = df.drop(columns=['salary']).astype(int)
X = sm.add_constant(X)
y = df['salary']

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 salary   R-squared:                       0.547
Model:                            OLS   Adj. R-squared:                  0.534
Method:                 Least Squares   F-statistic:                     40.91
Date:                Sat, 05 Apr 2025   Prob (F-statistic):          1.34e-241
Time:                        17:47:03   Log-Likelihood:                -20557.
No. Observations:                1673   AIC:                         4.121e+04
Df Residuals:                    1624   BIC:                         4.148e+04
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
                                                     coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------

In [203]:
# по одному удаляем незначимые признаки и пересчитываем модель
def iterative_feature_selection(X, y, threshold=0.05):
    while True:
        model1 = sm.OLS(y, X).fit()
        p_values = model1.pvalues.drop('const', errors='ignore')
        if all(p_values <= threshold):
            break
        worst_feature = p_values.idxmax()
        X = X.drop(columns=[worst_feature])
    return model1

final_model = iterative_feature_selection(X, y)

In [205]:
print(final_model.summary())

                            OLS Regression Results                            
Dep. Variable:                 salary   R-squared:                       0.543
Model:                            OLS   Adj. R-squared:                  0.537
Method:                 Least Squares   F-statistic:                     88.99
Date:                Sat, 05 Apr 2025   Prob (F-statistic):          4.87e-261
Time:                        17:52:32   Log-Likelihood:                -20566.
No. Observations:                1673   AIC:                         4.118e+04
Df Residuals:                    1650   BIC:                         4.130e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co

In [207]:
print(f"Старый R²: {model.rsquared:.3f}, Новый R²: {final_model.rsquared:.3f}")

Старый R²: 0.547, Новый R²: 0.543


In [209]:
print(f"Старый AIC: {model.aic:.1f}, Новый AIC: {final_model.aic:.1f}")

Старый AIC: 41212.7, Новый AIC: 41178.0


In [ ]:
df = df.drop(columns=['skill_HR-аналитика', 'skill_BPMN', 'skill_User Story', ])